# 4. Analyzing TAD conservation with BLAST

# Strategy 4
## Detecting conserved TADs with SVs
## Detecting split/merge events
### Finding % identity of NPB TADs in Azucena
1. Blasting all NPB TADs to azucena TADs with no min perc_ident or alignment rate, no max_target_seqs
2. Extracting all hits per TAD (1 TAD - 1 target), summarizing, recording query coverage
3. Identifying average/median query_cov, plotting their distribution

In [1]:
#1. Extracting TAD sequences to x_tads.fna
module load bedtools/intel/2.29.2
bedtools getfasta -fi ../genomes/NPB.fna -bed ../comparative_TADs_boundaries/NPB_TADs_ranked.bed -fo npb_tads.fna
bedtools getfasta -fi ../genomes/azucena.fna -bed ../comparative_TADs_boundaries/az_TADs_5kb_two_tools_80.bed -fo az_tads.fna
bedtools getfasta -fi ../genomes/IR64.fna -bed ../comparative_TADs_boundaries/IR64_TADs_5kb_two_tools_80.bed -fo ir64_tads.fna
bedtools getfasta -fi ../genomes/orufi.fna -bed ../comparative_TADs_boundaries/oruf_TADs_5kb_two_tools_80.bed -fo oruf_tads.fna
bedtools getfasta -fi ../genomes/omer.fna -bed ../comparative_TADs_boundaries/omer_TADs_5kb_two_tools_80.bed -fo omer_tads.fna

index file ../genomes/azucena.fna.fai not found, generating...
index file ../genomes/IR64.fna.fai not found, generating...
index file ../genomes/orufi.fna.fai not found, generating...
index file ../genomes/omer.fna.fai not found, generating...


In [1]:
#2. npb_az_all_blast_results4.txt
#query = npb
#targets = az,ir64,oruf,omer
cd ../Method_4/
sbatch blast.sh

Submitted batch job 40677642


In [5]:
#filter the results so that only TADs on the same chr in query and target remain, and the length of hsp alignment is minimum 5000
import pandas as pd

# Read the input file
file_path = 'npb_az_all_blast_results4.txt'
df = pd.read_table(file_path, header=None, names=['col1', 'col2', 'identity', 'length', 'mismatches', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e_value', 'bit_score', 'alignment_length'])

# Extract numbers after 'chr' in col1 and col2
df['chr1_number'] = df['col1'].str.extract(r'chr(\d+)')
df['chr2_number'] = df['col2'].str.extract(r'chr(\d+)')

# Filter rows based on conditions
filtered_df = df[(df['chr1_number'] == df['chr2_number']) & (df['length'] >= 5000)]

# Drop the temporary columns
filtered_df = filtered_df.drop(['chr1_number', 'chr2_number'], axis=1)

# Save the filtered DataFrame to a new file
filtered_file_path = 'npb_az_all_blast_results4_filtered.txt'
filtered_df.to_csv(filtered_file_path, sep='\t', index=False, header=False)

In [8]:
#3 Per NPB_TAD, report all az_TADs and their cumulative coverages
# output file structure: npb_TAD identifier, 
#                         az_TAD-n identifier
#                         Qlen (col13) TAD length in NPB
#                         cumulative Tlength (az_TAD-n length) = sum of col4 values per entry in col1+col2 of input file
#                         percentage Tlen/Qlen
                        
import pandas as pd

# Read the input file
file_path = 'npb_az_all_blast_results4_filtered.txt'
df = pd.read_table(file_path, header=None, names=['col1', 'col2', 'identity', 'length', 'mismatches', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e_value', 'bit_score', 'alignment_length'])

# Create a column for sorting based on the original order
df['sort_order'] = range(len(df))

# Group by col1 and col2, and calculate the sum of col4 for each group
grouped_df = df.groupby(['col1', 'col2'], as_index=False)['length'].sum()

# Merge with the original DataFrame to get the last col value
summary_df = pd.merge(df[['col1', 'col2', 'alignment_length', 'sort_order']], grouped_df, on=['col1', 'col2'])

# Select the first row of each group
summary_df = summary_df.groupby(['col1', 'col2']).first().reset_index()

# Sort the summary DataFrame based on the original order
summary_df = summary_df.sort_values(by='sort_order').drop('sort_order', axis=1)

# Calculate col5
summary_df['col5'] = (summary_df['length'] / summary_df['alignment_length']) * 100

# Save the summary DataFrame to a new file
summary_file_path = 'npb_az_summary4.txt'
summary_df.to_csv(summary_file_path, sep='\t', index=False, header=False)

In [2]:
#For each NPB_TAD, I will check all az_TADs 
#If the az_TAD-2 start is farther than 100kb from az_TAD1 end, it will be discarded
#continue until all az_TADs processed

In [51]:
#create a library (key = npb_tad, value = list of az_tads)
def create_dict_from_file(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            key = parts[0]
            value = parts[1]

            if key not in result_dict:
                result_dict[key] = [value]
            else:
                result_dict[key].append(value)

    return result_dict

# Example usage
file_path = 'npb_az_summary4.txt'
result_dict = create_dict_from_file(file_path)

# Print the first three key-value combinations
count = 0
for key, values in result_dict.items():
    print(f"{key}: {values}")
    count += 1
    if count == 5:
        break

chr01:150000-210000: ['chr01:130000-195000']
chr01:420000-525000: ['chr01:21820000-21915000', 'chr01:85000-130000', 'chr01:2740000-2805000', 'chr01:23985000-24035000', 'chr01:38075000-38180000', 'chr01:7105000-7185000', 'chr01:2805000-2940000', 'chr01:23630000-23725000', 'chr01:10245000-10295000', 'chr01:15870000-15990000']
chr01:2640000-2695000: ['chr01:2740000-2805000', 'chr01:2805000-2940000']
chr01:2695000-2810000: ['chr01:2805000-2940000']
chr01:2890000-2965000: ['chr01:15870000-15990000', 'chr01:10295000-10400000', 'chr01:29350000-29450000', 'chr01:21820000-21915000', 'chr01:23985000-24035000']


In [59]:
#create function to filter az_tads for each npb_tad, so that the az_tads are not farther than 100kb from each other
def filter_values(dictionary):
    filtered_dict = {}

    for key, values in dictionary.items():
        if len(values) == 1:
            # If there is only one value, keep it as is
            filtered_dict[key] = values
        else:
            # If there are multiple values, process them
            filtered_values = [values[0]]  # Keep the first value
    
            for i in range(1, len(values)):
                # Split each value by ':' and '-'
                parts1 = values[i - 1].split(':')[-1].split('-')
                parts1.insert(0, values[i - 1].split(':')[0])

                parts2 = values[i].split(':')[-1].split('-')
                parts2.insert(0, values[i].split(':')[0])
                
                # Compare the parts and apply the filtering conditions
                if parts1[0] == parts2[0] and abs(int(parts2[1]) - int(parts1[2])) < 100000:
                    filtered_values.append(values[i])
                    
            filtered_dict[key] = filtered_values

    return filtered_dict

In [60]:
# Assuming `result_dict` is the initial dictionary
filtered_dict = filter_values(result_dict)

# Print the first three key-value combinations
count = 0
for key, values in filtered_dict.items():
    print(f"{key}: {values}")
    count += 1
    if count == 5:
        break

chr01:150000-210000: ['chr01:130000-195000']
chr01:420000-525000: ['chr01:21820000-21915000']
chr01:2640000-2695000: ['chr01:2740000-2805000', 'chr01:2805000-2940000']
chr01:2695000-2810000: ['chr01:2805000-2940000']
chr01:2890000-2965000: ['chr01:15870000-15990000']


In [1]:
#calculate number of conserved TADs from npb_az_summary4_filtered.txt
#conserved would be rows with unique entries in both col1 and col2
import csv

def save_conserved_rows(input_file, output_file):
    col1_count = {}
    col2_count = {}
    conserved_rows = []

    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        header = next(reader, None)  # Read the header if present
        if header:
            conserved_rows.append(header)

        for row in reader:
            col1_value = row[0]
            col2_value = row[1]

            # Check uniqueness in col1
            if col1_value not in col1_count:
                col1_count[col1_value] = 1
            else:
                col1_count[col1_value] += 1

            # Check uniqueness in col2
            if col2_value not in col2_count:
                col2_count[col2_value] = 1
            else:
                col2_count[col2_value] += 1

            # If both col1 and col2 are unique, add the row to conserved_rows
            if col1_count[col1_value] == 1 and col2_count[col2_value] == 1:
                conserved_rows.append(row)

    # Write the conserved rows to the output file
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output, delimiter='\t')
        writer.writerows(conserved_rows)

# Example usage
input_file = 'npb_az_summary4_filtered.txt'
output_file = 'npb_az_conserved.txt'
save_conserved_rows(input_file, output_file)

In [15]:
### How many TADs are conserved NPB-Az
# Read the data from the file
data = pd.read_csv("npb_az_conserved.txt", sep='\t', header=None, names=['npb_tad', 'az_tad', 'npb_len', 'az_len', 'coverage'])

    # Calculate and print mean coverage
mean_coverage = data['coverage'].mean()
print(f"Mean coverage: {mean_coverage:.2f}")

    # Calculate and print median coverage
median_coverage = data['coverage'].median()
print(f"Median coverage: {median_coverage:.2f}")

    # Calculate and print total number of rows
total_rows = len(data)
print(f"Total conserved TADs: {total_rows}")

    # Calculate and print proportion of rows with coverage >= 50, 80, 90, 100
coverage_above_50 = data[data['coverage'] >= 50]
coverage_above_80 = data[data['coverage'] >= 80]
coverage_above_90 = data[data['coverage'] >= 90]
coverage_above_100 = data[data['coverage'] >= 100]
    
    # Calculate and print number of rows with coverage >= 50, 80, 90, 100
print(f"Coverage >= 50: {len(coverage_above_50)} TADs")
print(f"Coverage >= 80: {len(coverage_above_80)} TADs")
print(f"Coverage >= 90: {len(coverage_above_90)} TADs")
print(f"Coverage >= 100: {len(coverage_above_100)} TADs\n")

#compare with results from Method3
# Coverage >= 50: 555 TADs
# Coverage >= 80: 486 TADs
# Coverage >= 90: 438 TADs
# Coverage >= 100: 305 TADs
#the difference is due to filtering: length of hsp alignment is minimum 5000 in Method4, any in Method3
#I'm gonna use results of Method3 for conserved TADs annotation

Mean coverage: 66.62
Median coverage: 85.82
Total conserved TADs: 621
Coverage >= 50: 427 TADs
Coverage >= 80: 352 TADs
Coverage >= 90: 264 TADs
Coverage >= 100: 55 TADs



In [8]:
#detecting split events and saving to npb_az_splits.txt
#col1=1, col2>=2, sum(col4)>=50

import csv

def detect_and_save_events(input_file, output_file):
    # Load the input file into a list of lists
    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        data = list(reader)

    # Create a dictionary to store the count of unique strings from col1
    col1_count = {}

    # Create a list to store rows that satisfy the conditions
    filtered_rows = []

    # Iterate through the rows in the data
    for row in data:
        col1_value = row[0]  # Assuming col1 is at index 0
        col2_value = row[1]  # Assuming col2 is at index 1
        col4_value = int(row[3])  # Assuming col4 is at index 3

        # Update col1_count dictionary
        if col1_value not in col1_count:
            col1_count[col1_value] = {col2_value}
        else:
            col1_count[col1_value].add(col2_value)

        # Check conditions for detecting events
        if len(col1_count[col1_value]) >= 2 and sum(int(row[3]) for row in data if row[0] == col1_value) >= 0:
            filtered_rows.extend([row for row in data if row[0] == col1_value])

    # Write the filtered rows to the output file
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows(filtered_rows)

    # Return the number of events
    num_events = sum(1 for col1_value, count in col1_count.items() if len(count) >= 2 and sum(int(row[3]) for row in data if row[0] == col1_value) >= 50)
    return num_events

# Example usage
input_file = 'npb_az_summary4_filtered.txt'
output_file = 'npb_az_splits.txt'
num_events = detect_and_save_events(input_file, output_file)

print(f"Number of split events: {num_events}")

Number of split events: 42


In [9]:
#detecting merge events and saving to npb_az_merges.txt
#col1>=2, col2=1, sum(col4)>=50

import csv

def detect_and_save_events(input_file, output_file):
    # Load the input file into a list of lists
    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        data = list(reader)

    # Create a dictionary to store the count of unique strings from col2
    col2_count = {}

    # Create a list to store rows that satisfy the conditions
    filtered_rows = []

    # Iterate through the rows in the data
    for row in data:
        col1_value = row[0]  # Assuming col1 is at index 0
        col2_value = row[1]  # Assuming col2 is at index 1
        col4_value = int(row[3])  # Assuming col4 is at index 3

        # Update col2_count dictionary
        if col2_value not in col2_count:
            col2_count[col2_value] = {col1_value}
        else:
            col2_count[col2_value].add(col1_value)

        # Check conditions for detecting events
        if len(col2_count[col2_value]) >= 2 and sum(int(row[3]) for row in data if row[1] == col2_value) >= 0:
            filtered_rows.extend([row for row in data if row[1] == col2_value])

    # Write the filtered rows to the output file
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows(filtered_rows)

    # Return the number of events
    num_events = sum(1 for col2_value, count in col2_count.items() if len(count) >= 2 and sum(int(row[3]) for row in data if row[1] == col2_value) >= 50)
    return num_events

# Example usage
input_file = 'npb_az_summary4_filtered.txt'
output_file = 'npb_az_merges_draft.txt'
num_events = detect_and_save_events(input_file, output_file)

print(f"Number of merge events (non-filtered for distance): {num_events}")

Number of merge events (non-filtered for distance): 134


In [16]:
#filtering the merge events from npb_az_merges_draft.txt so that the distance between npb_tads is <=100kb
#create a library (key = az_tad, value = list of npb_tads)
def create_dict_from_file(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            key = parts[1]
            value = parts[0]

            if key not in result_dict:
                result_dict[key] = [value]
            else:
                # Add the value only if it's not already in the list
                if value not in result_dict[key]:
                    result_dict[key].append(value)

    return result_dict

# Example usage
file_path = 'npb_az_merges_draft.txt'
result_dict = create_dict_from_file(file_path)

# Print the first three key-value combinations
count = 0
for key, values in result_dict.items():
    print(f"{key}: {values}")
    count += 1
    if count == 3:
        break

chr01:2805000-2940000: ['chr01:2640000-2695000', 'chr01:2695000-2810000', 'chr01:14470000-14515000']
chr01:9435000-9540000: ['chr01:9115000-9170000', 'chr01:9170000-9275000', 'chr01:34550000-34635000']
chr01:5660000-5730000: ['chr01:5480000-5545000', 'chr01:10900000-10985000']


In [17]:
#create function to filter npb_tads for each az_tad, so that the npb_tads are not farther than 100kb from each other
def filter_values(dictionary):
    filtered_dict = {}

    for key, values in dictionary.items():
        if len(values) == 1:
            # If there is only one value, keep it as is
            filtered_dict[key] = values
        else:
            # If there are multiple values, process them
            filtered_values = [values[0]]  # Keep the first value
    
            for i in range(1, len(values)):
                # Split each value by ':' and '-'
                parts1 = values[i - 1].split(':')[-1].split('-')
                parts1.insert(0, values[i - 1].split(':')[0])

                parts2 = values[i].split(':')[-1].split('-')
                parts2.insert(0, values[i].split(':')[0])
                
                # Compare the parts and apply the filtering conditions
                if parts1[0] == parts2[0] and abs(int(parts2[1]) - int(parts1[2])) < 100000:
                    filtered_values.append(values[i])
                    
            filtered_dict[key] = filtered_values

    return filtered_dict

# Assuming `result_dict` is the initial dictionary
filtered_dict = filter_values(result_dict)

# Print the first three key-value combinations
count = 0
for key, values in filtered_dict.items():
    print(f"{key}: {values}")
    count += 1
    if count == 5:
        break

chr01:2805000-2940000: ['chr01:2640000-2695000', 'chr01:2695000-2810000']
chr01:9435000-9540000: ['chr01:9115000-9170000', 'chr01:9170000-9275000']
chr01:5660000-5730000: ['chr01:5480000-5545000']
chr01:10475000-10545000: ['chr01:10195000-10265000']
chr01:22885000-22960000: ['chr01:14835000-14995000', 'chr01:22260000-22335000']


In [19]:
#create a npb_az_merges.txt from filtered_dict !!! this file does not represent merges only !!!
#for each az_tad, I have npb_tads that it is split into
#order of cols: npb_tad, az_tad, etc

import csv

# Load the contents of npb_az_merges_draft.txt into a list of lines
with open('npb_az_merges_draft.txt', 'r') as file:
    summary_lines = file.readlines()

# Create a list to store the filtered lines
filtered_lines = []

# Iterate through the entries in the filtered_dict
for key, values in filtered_dict.items():
    # Iterate through the values for each key
    for value in values:
        # Check if the entry (key + '\t' + value) is present in npb_az_merges_draft.txt
        entry = f"{value}\t{key}"
        for line in summary_lines:
            if entry + '\t' in line:
                # If the entry is found, add the entire row to the filtered lines list
                filtered_lines.append(line.strip() + '\n')  # Add a newline character
                break  # Stop searching for the same entry in subsequent lines

# Write the filtered lines to npb_az_summary4_filtered.txt
with open('npb_az_merges.txt', 'w', newline='') as file:
    file.writelines(filtered_lines)

In [3]:
import csv
from collections import Counter

def extract_repeated_rows(input_file, output_file):
    col2_counts = Counter()

    # Read the file and count occurrences of values in col2
    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        header = next(reader)  # Read and store the header
        col2_index = header.index("Col2") if "Col2" in header else 1

        for row in reader:
            col2_value = row[col2_index]
            col2_counts[col2_value] += 1

    # Identify values in col2 that are repeated more than once
    repeated_values = {key for key, count in col2_counts.items() if count > 1}

    # Write rows containing repeated values in col2 to the output file
    with open(input_file, 'r', newline='') as file, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(file, delimiter='\t')
        writer = csv.writer(outfile, delimiter='\t')
        writer.writerow(header)  # Write header

        for row in reader:
            col2_value = row[col2_index]
            if col2_value in repeated_values:
                writer.writerow(row)

# Replace 'npb_az_merges.txt' and 'npb_az_merges_true.txt' with your actual file names
input_file = 'npb_az_merges.txt'
output_file = 'npb_az_merges_true.txt'
extract_repeated_rows(input_file, output_file)

In [4]:
#create file npb_az_merges_true with real merge events
import csv

def write_filtered_entries(input_file, output_file):
    col2_entries = {}

    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Skip header if present
        for row in reader:
            col1_value = row[0]  # Assuming col1 is at index 0
            col2_value = row[1]  # Assuming col2 is at index 1

            if col2_value not in col2_entries:
                col2_entries[col2_value] = {col1_value}
            else:
                col2_entries[col2_value].add(col1_value)

    # Write unique entries in col2 with two or more unique entries in col1 to the output file
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter='\t')
        writer.writerow(["Col1", "Col2"])  # Write header

        for col2_value, col1_set in col2_entries.items():
            if len(col1_set) >= 2:
                for col1_value in col1_set:
                    writer.writerow([col1_value, col2_value])

input_file = 'npb_az_merges.txt'
output_file = 'filtered_entries.txt'
write_filtered_entries(input_file, output_file)

In [2]:
#calculate number of merge events (2+ npb_tads into 1 az_tad) filtered for distance (<=100kb)
import csv

def count_events(input_file):
    col2_count = {}

    with open(input_file, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)  # Skip header if present
        for row in reader:
            col1_value = row[0]  # Assuming col1 is at index 0
            col2_value = row[1]  # Assuming col2 is at index 1

            if col2_value not in col2_count:
                col2_count[col2_value] = {col1_value}
            else:
                col2_count[col2_value].add(col1_value)

    # Count the number of unique entries in col2 with two or more unique entries in col1
    num_events = sum(1 for count in col2_count.values() if len(count) >= 2)

    return num_events

input_file = 'npb_az_merges.txt'
num_events = count_events(input_file)

print(f"Number of merge events: {num_events}")

Number of merge events: 47


### Nipponbare-Azucena comparison
1. How many split events happened? (1 NPB TAD split into 2/3/n Azucena TADs) 42
2. How many merge events happened? (2/3/n NPB TADs merged into 1 Azucena TAD) 47 (merged TADs = 94)
3. How many complex rearrangements happened? (Split+merge event) N/A
4. How many NPB TADs remained conserved in Azucena (% conservation >=50) (Use info from Method 3, cov>=50%) 555

In [ ]:
split event
1 npb_tad = 2+ az_tads AND sum(Tlengths/Qlengths from last col)>=50
#actually if sum>= 0 the results is the same
merge event
2+ npb_tad = 1 az_tad AND sum(Tlengths/Qlengths)>=50
#actually if sum>= 0 the results is the same
conserved
1 npb_tad = 1 az_tad AND sum(Tlengths/Qlengths)>=50
#took from the Method 3 results
complex event
how to define???